## Forced particle on a circle

Pierre de Buyl  
Instituut voor Theoretische Fysica, KU Leuven

The code and notebooks in the repository `2018_nonequilibrium_simulations`
constitute supplementary material for the lecture notes
*Langevin simulations for nonequilibrium physics*.
See the [README.md](README.md) file for more information.
See the lecture notes (link posted soon, visit [my website](http://pdebuyl.be/)
for updates.

This notebook contains the example of a colloidal particle in a toroidal trap.

In [ ]:
# import libraries and set default figure parameters
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import algorithms
import math

In [ ]:
plt.rcParams['figure.figsize'] = (5,3)
plt.rcParams['figure.subplot.top'] = 0.95
plt.rcParams['figure.subplot.right'] = 0.95

plt.rcParams['font.size'] = 16

In [ ]:
%load_ext cython

In [ ]:

def U(theta, A):
    return A*np.sin(theta)

def f(theta, A):
    return -A*np.cos(theta)

def fixed_A_force(A):
    def force(theta):
        return f(theta, A)
    return force


In [ ]:
%%cython

cimport algorithms
from libc.math cimport cos

cdef class cy_A_force(algorithms.cyfunc_nd):
    cdef double A
    def __init__(self, A):
        self.A = A
    cpdef void force(self, double[::1] x, double[::1] f):
        cdef int i
        for i in range(x.shape[0]):
            f[i] = -self.A*cos(x[i])

cdef class cy_A_drive(algorithms.cyfunc_nd):
    cdef double A
    cdef double F
    def __init__(self, A, F):
        self.A = A
        self.F = F
    cpdef void force(self, double[::1] x, double[::1] f):
        cdef int i
        for i in range(x.shape[0]):
            f[i] = -self.A*cos(x[i]) + self.F


In [ ]:
mu = np.array([1], dtype=float)
T = 2
dt = 1e-2
nsteps = 50
npoints = 50000

A = 0.87
F = 0.4

theta = np.array([0], dtype=float)

theta = algorithms.integrate_srk(theta, mu, T, dt, npoints, nsteps, cy_A_drive(A, F))


In [ ]:
periodic_theta = np.mod(theta, 2*np.pi)
count, bins, patches = plt.hist(periodic_theta, density=True, bins=32, histtype='step');

thr = np.linspace(0, 2*np.pi, 100)
p = np.exp(-U(thr, A)/T)
p /= np.sum(p)*(thr[1]-thr[0])
plt.plot(thr, p)

#plt.plot(thr, U(thr, A))

In [ ]:
plt.plot(theta)

In [ ]:
mid_bins = (bins[1:]+bins[:-1])/2
np.sum(U(mid_bins, A)*count)*(bins[1]-bins[0])

In [ ]:
np.mean(U(theta, A))

In [ ]:
mu = np.array([1], dtype=float)
T = 1
dt = 1e-4
nsteps = 10
npoints = 400000

delta_A = 0.1
xi_data = []
for i in range(40):

    A = 1
    F = 0.5

    theta = np.array([0], dtype=float)

    theta = algorithms.integrate_euler(theta, mu, T, dt, npoints, nsteps, cy_A_drive(A, F))
    A_plus = 1 + delta_A
    
    theta_perturbed = algorithms.integrate_srk(theta[-1], mu, T, dt, 400, nsteps, cy_A_drive(A_plus, F))
    U0_perturbed = U(theta_perturbed[0], A_plus)
    print(U0_perturbed)
    if np.abs(U0_perturbed) > 0.95*A:
        print('drop')
        continue
        
    theta_unperturbed = algorithms.integrate_srk(theta[-1], mu, T, dt, npoints, nsteps, cy_A_drive(A, F))
    
    starting_point = np.argwhere(np.abs(U(theta_unperturbed, A) - U0_perturbed) < 1e-2)[0][0]
    if starting_point < npoints - 400: 
        xi_data.append(U(theta_perturbed, A_plus)-U(theta_unperturbed[starting_point:starting_point+400], A))

xi_data = np.array(xi_data)/-delta_A

In [ ]:
plt.figure()
time = np.arange(xi_data.shape[1])*dt*nsteps
plt.plot(time, xi_data.mean(axis=0))

In [ ]:
C = (U(theta_unperturbed, A)**2).mean() - tidynamics.acf(U(theta_unperturbed, A))

In [ ]:

K_term = U(theta_unperturbed, A) + (F-A**2*np.cos(theta_unperturbed))*A*np.cos(theta_unperturbed)

K_cor = tidynamics.core.correlation_1d(K_term, U(theta_unperturbed, A))[len(theta_unperturbed)-1:]

K = -scipy.integrate.cumtrapz(K_cor)*nsteps*dt

In [ ]:
import scipy.integrate


In [ ]:
plt.figure()
plt.plot(K[:100])
plt.plot(C[:100])
plt.plot((K[:100]+C[:100])/2)
plt.plot(xi_data.mean(axis=0)[:100])